In [1]:
import os
import requests
import pandas as pd
import gspread

from datetime import datetime
from pathlib import Path
from dotenv import load_dotenv
from gspread_dataframe import set_with_dataframe
from oauth2client.service_account import ServiceAccountCredentials
from urllib.parse import quote

# === Descobre o caminho absoluto seguro do .env ===
try:
    # Scripts (.py)
    base_dir = Path(__file__).resolve().parent
except NameError:
    # Notebooks (Jupyter ou Papermill)
    base_dir = Path().resolve()

dotenv_path = Path("C:/Users/Camilo_Bica/data_science/consultoria/escola_policia/secrets/.env")

# === Log de carregamento ===
print(f"🔍 Carregando .env de: {dotenv_path}")
loaded = load_dotenv(dotenv_path, override=True)
print(f"✅ .env carregado com sucesso? {loaded}")

# === Carrega e valida as variáveis de ambiente ===
API_URL = os.getenv("API_URL")
API_KEY = os.getenv("API_KEY")

print(f"🔑 API_KEY: {'(carregada)' if API_KEY else '❌ NÃO CARREGADA'}")

assert API_URL is not None and API_URL.startswith("http"), "❌ API_URL não carregada corretamente!"
assert API_KEY is not None, "❌ API_KEY não carregada corretamente!"

HEADERS = {"Api-Token": API_KEY}

# === Testa conexão com a API ===
def testar_conexao():
    try:
        url = f"{API_URL}/contacts?limit=1"
        response = requests.get(url, headers=HEADERS)
        response.raise_for_status()
        print("✅ Conexão com a API bem-sucedida.")
        contato_teste = response.json().get("contacts", [{}])[0]
        print("Exemplo de contato:", contato_teste.get("email", "sem email"))
    except requests.exceptions.RequestException as e:
        print("❌ Falha na conexão com a API:", e)
        print("🔎 URL testada:", url)
    except Exception as e:
        print("❌ Erro inesperado:", e)

# Executa teste
testar_conexao()

🔍 Carregando .env de: C:\Users\Camilo_Bica\data_science\consultoria\escola_policia\secrets\.env
✅ .env carregado com sucesso? True
🔑 API_KEY: (carregada)
✅ Conexão com a API bem-sucedida.
Exemplo de contato: *gilbertogallo82@gmail.com


In [2]:
requests.get(f"{API_URL}/fields", headers=HEADERS)

<Response [200]>

In [3]:
# Buscar todos os campos personalizados
response = requests.get(f"{API_URL}/fields", headers=HEADERS)
response.raise_for_status()

campos = response.json().get("fields", [])

print("\n=== Campos Personalizados ===")
for campo in campos:
    print(f"ID: {campo['id']} - Nome: {campo['title']}")


=== Campos Personalizados ===
ID: 6 - Nome: estado
ID: 7 - Nome: idade
ID: 8 - Nome: escolaridade
ID: 9 - Nome: renda
ID: 10 - Nome: estado civil
ID: 11 - Nome: filhos
ID: 12 - Nome: escolheu profissão
ID: 13 - Nome: dificuldade
ID: 14 - Nome: email captação
ID: 15 - Nome: telefone captação
ID: 18 - Nome: data inscrição lançamento
ID: 24 - Nome: utm_source
ID: 25 - Nome: utm_campaign
ID: 26 - Nome: utm_medium
ID: 27 - Nome: utm_content
ID: 28 - Nome: utm_term


In [4]:
# === Caminho fixo para salvar parquet
output_dir = Path("C:/Users/Camilo_Bica/data_science/consultoria/escola_policia/dados")
output_dir.mkdir(parents=True, exist_ok=True)
parquet_path = output_dir / "leads_totais.parquet"
coluna_data = "data inscrição lançamento"

# === Função para buscar todos os contatos + fieldValues com filtro incremental
def buscar_todos_contatos_incremental(api_url, headers, data_inicio=None):
    contatos = []
    field_values = []
    offset = 0
    limit = 100
    pagina = 1

    filtro_data = ""
    if data_inicio is not None and not pd.isna(data_inicio):
        data_formatada = quote(data_inicio.strftime("%Y-%m-%dT%H:%M:%S-03:00"))
        filtro_data = f"&filters[created_after]={data_formatada}"
        print(f"📆 Buscando contatos após: {data_inicio}")
    else:
        print("⚠️ Nenhuma data válida encontrada. Coletando tudo.")

    while True:
        url = f"{api_url}/contacts?limit={limit}&offset={offset}&include=fieldValues{filtro_data}"
        print(f"🔄 Página {pagina} | Offset {offset} | Coletando...")

        resp = requests.get(url, headers=headers)
        resp.raise_for_status()
        data = resp.json()

        novos_contatos = data.get("contacts", [])
        novos_fields = data.get("fieldValues", [])

        if not novos_contatos:
            print("✅ Fim da paginação: sem novos contatos.")
            break

        contatos.extend(novos_contatos)
        field_values.extend(novos_fields)

        print(f"📦 +{len(novos_contatos)} contatos acumulados: {len(contatos)} total")

        offset += limit
        pagina += 1

    print(f"✅ Coleta finalizada com {len(contatos)} novos contatos.")
    return contatos, field_values

# === Buscar campos personalizados
def buscar_campos_personalizados(api_url, headers):
    resp = requests.get(f"{api_url}/fields", headers=headers)
    resp.raise_for_status()
    return {
        str(f["id"]): f["title"].strip()
        for f in resp.json().get("fields", [])
    }

# === Etapa 1: carregar histórico existente
if parquet_path.exists():
    df_existente = pd.read_parquet(parquet_path)
    print(f"📂 Histórico carregado: {len(df_existente)} registros.")

    if coluna_data in df_existente.columns:
        df_existente[coluna_data] = pd.to_datetime(df_existente[coluna_data], errors="coerce")
        ultima_data = df_existente[coluna_data].max()

        if pd.isna(ultima_data):
            print("⚠️ Nenhuma data válida encontrada na coluna. Coletando tudo.")
            ultima_data = None
        else:
            ultima_data -= pd.Timedelta(minutes=1)
            print(f"📅 Última data de cadastro registrada: {ultima_data}")
    else:
        print(f"⚠️ Coluna '{coluna_data}' não encontrada no Parquet. Coletando tudo.")
        ultima_data = None
else:
    df_existente = pd.DataFrame()
    ultima_data = None
    print("📂 Nenhum histórico encontrado. Iniciando coleta completa.")

# === Etapa 2: executar coleta incremental
contatos, field_values = buscar_todos_contatos_incremental(API_URL, HEADERS, ultima_data)
campos_personalizados = buscar_campos_personalizados(API_URL, HEADERS)

# === Etapa 3: mapear campos personalizados
campos_por_contato = {}
for fv in field_values:
    cid = fv["contact"]
    campo = campos_personalizados.get(str(fv["field"]), f"custom_{fv['field']}")
    campos_por_contato.setdefault(cid, {})[campo] = fv["value"]

# === Etapa 4: montar DataFrame com os dados
dados_contatos = []
for contato in contatos:
    cid = contato["id"]
    campos = campos_por_contato.get(cid, {})
    dados = {
        "id": cid,
        "nome": contato.get("firstName", ""),
        "email": contato.get("email", ""),
        "telefone": contato.get("phone", ""),
        "data": pd.to_datetime(campos.get("data inscrição lançamento", ""), errors="coerce")
    }
    dados.update(campos)
    dados_contatos.append(dados)

df_todos_contatos = pd.DataFrame(dados_contatos)

# === Etapa 5: consolidar e salvar no Parquet
if not df_todos_contatos.empty:
    df_existente[coluna_data] = pd.to_datetime(df_existente[coluna_data], errors="coerce")
    df_todos_contatos[coluna_data] = pd.to_datetime(df_todos_contatos[coluna_data], errors="coerce")
    df_todos_contatos = pd.concat([df_existente, df_todos_contatos], ignore_index=True)
    df_todos_contatos = df_todos_contatos.drop_duplicates(subset=["id"], keep="last")
    df_todos_contatos.to_parquet(parquet_path, index=False)
    print(f"💾 Parquet atualizado: {len(df_todos_contatos)} registros salvos.")
else:
    print("⚠️ Nenhum novo contato encontrado. Parquet mantido como está.")

📂 Histórico carregado: 29788 registros.
📅 Última data de cadastro registrada: 2025-05-28 23:59:00
📆 Buscando contatos após: 2025-05-28 23:59:00
🔄 Página 1 | Offset 0 | Coletando...
📦 +100 contatos acumulados: 100 total
🔄 Página 2 | Offset 100 | Coletando...
📦 +100 contatos acumulados: 200 total
🔄 Página 3 | Offset 200 | Coletando...
📦 +100 contatos acumulados: 300 total
🔄 Página 4 | Offset 300 | Coletando...
📦 +100 contatos acumulados: 400 total
🔄 Página 5 | Offset 400 | Coletando...
📦 +100 contatos acumulados: 500 total
🔄 Página 6 | Offset 500 | Coletando...
📦 +100 contatos acumulados: 600 total
🔄 Página 7 | Offset 600 | Coletando...
📦 +100 contatos acumulados: 700 total
🔄 Página 8 | Offset 700 | Coletando...
📦 +100 contatos acumulados: 800 total
🔄 Página 9 | Offset 800 | Coletando...
📦 +100 contatos acumulados: 900 total
🔄 Página 10 | Offset 900 | Coletando...
📦 +100 contatos acumulados: 1000 total
🔄 Página 11 | Offset 1000 | Coletando...
📦 +100 contatos acumulados: 1100 total
🔄 Pági

In [5]:
df_todos_contatos = df_todos_contatos.drop(columns=["data"])

In [6]:
df_leads_l34 = df_todos_contatos[df_todos_contatos["data inscrição lançamento"].notna()].copy()

In [7]:
df_leads_l34.info()

<class 'pandas.core.frame.DataFrame'>
Index: 28278 entries, 0 to 33119
Data columns (total 20 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   id                         28278 non-null  object        
 1   nome                       28278 non-null  object        
 2   email                      28278 non-null  object        
 3   telefone                   28278 non-null  object        
 4   utm_source                 27590 non-null  object        
 5   utm_campaign               27588 non-null  object        
 6   utm_medium                 26642 non-null  object        
 7   utm_content                25989 non-null  object        
 8   utm_term                   23412 non-null  object        
 9   data inscrição lançamento  28278 non-null  datetime64[ns]
 10  estado                     22939 non-null  object        
 11  idade                      22552 non-null  object        
 12  escolarid

In [8]:
def carregar_csv(caminho_arquivo):
    caminho = Path(caminho_arquivo)
    if not caminho.exists():
        print(f"[ERRO] Arquivo CSV não encontrado: {caminho.resolve()}")
        return None

    for sep in [',', ';', '\t', '|']:
        try:
            df = pd.read_csv(caminho, sep=sep, engine='python')
            print(f"[OK] CSV carregado com separador '{sep}': {caminho.name}")
            return df
        except Exception as e:
            print(f"[INFO] Tentativa com separador '{sep}' falhou: {e}")

    print(f"[ERRO] Falha ao carregar CSV '{caminho.name}' com os separadores testados.")
    return None

# === Define caminho absoluto corretamente ===
try:
    base_dir = Path(__file__).resolve().parent
except NameError:
    base_dir = Path().resolve()

# Ajusta para garantir que você fique em .../escola_policia/
project_root = base_dir if base_dir.name == "escola_policia" else base_dir.parent
csv_elementor_path = project_root / "dados" / "elementor-submissions.csv"

# === Carregamento do CSV ===
df_elementor_leads = carregar_csv(csv_elementor_path)

[OK] CSV carregado com separador ',': elementor-submissions.csv


In [9]:
df_elementor_leads.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 516 entries, 0 to 515
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Nome            511 non-null    object 
 1   E-mail          516 non-null    object 
 2   Whatsapp        516 non-null    object 
 3   utm_medium      159 non-null    object 
 4   utm_campaign    159 non-null    object 
 5   utm_source      159 non-null    object 
 6   utm_term        159 non-null    object 
 7   utm_content     159 non-null    object 
 8   Form Name (ID)  516 non-null    object 
 9   Submission ID   516 non-null    int64  
 10  Created At      516 non-null    object 
 11  User ID         516 non-null    int64  
 12  User Agent      0 non-null      float64
 13  User IP         0 non-null      float64
 14  Referrer        516 non-null    object 
dtypes: float64(2), int64(2), object(11)
memory usage: 60.6+ KB


In [10]:
# Renomeia "E-mail" para "email" no df_elementor
df_elementor_leads.rename(columns={"E-mail": "email"}, inplace=True)

# Normaliza os e-mails para garantir matching
df_leads_l34["email"] = df_leads_l34["email"].str.strip().str.lower()
df_elementor_leads["email"] = df_elementor_leads["email"].str.strip().str.lower()

# Lista das colunas UTM
utms = ["utm_source", "utm_campaign", "utm_medium", "utm_content", "utm_term"]

# Reduz df_elementor_leads a email + utms, removendo duplicados
df_elementor_reduced = df_elementor_leads[["email"] + utms].drop_duplicates("email")

# Faz o merge, deixando pandas adicionar _x e _y
df_leads_l34 = pd.merge(df_leads_l34, df_elementor_reduced, on="email", how="left", suffixes=("_x", "_y"))

# Resolve duplicatas de UTM, preferindo os valores do Elementor (_y)
for utm in utms:
    df_leads_l34[utm] = df_leads_l34[f"{utm}_y"].combine_first(df_leads_l34.get(f"{utm}_x"))

# Remove colunas com sufixos
df_leads_l34.drop(columns=[f"{utm}_x" for utm in utms] + [f"{utm}_y" for utm in utms], inplace=True)

In [11]:
df_leads_l34.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28278 entries, 0 to 28277
Data columns (total 20 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   id                         28278 non-null  object        
 1   nome                       28278 non-null  object        
 2   email                      28278 non-null  object        
 3   telefone                   28278 non-null  object        
 4   data inscrição lançamento  28278 non-null  datetime64[ns]
 5   estado                     22939 non-null  object        
 6   idade                      22552 non-null  object        
 7   escolaridade               22569 non-null  object        
 8   renda                      22732 non-null  object        
 9   estado civil               22657 non-null  object        
 10  filhos                     22784 non-null  object        
 11  escolheu profissão         22844 non-null  object        
 12  difi

In [12]:
# Converter para datetime e manter apenas a data
df_leads_l34['data inscrição lançamento'] = pd.to_datetime(df_leads_l34['data inscrição lançamento'], errors='coerce').dt.date

# Regra 1: Facebook-Ads entre 17 e 18 de maio com utm_source ausente
mask_fb = (
    (df_leads_l34['data inscrição lançamento'] >= pd.to_datetime('2025-05-17').date()) &
    (df_leads_l34['data inscrição lançamento'] <= pd.to_datetime('2025-05-18').date()) &
    (df_leads_l34['utm_source'].isna())
)
df_leads_l34.loc[mask_fb, 'utm_source'] = 'Facebook-Ads'

# Regra 2: Google-Ads em 20 ou 21 de maio com utm_source ausente ou igual a 'tva'
mask_google = (
    df_leads_l34['data inscrição lançamento'].isin([pd.to_datetime('2025-05-20').date(), pd.to_datetime('2025-05-21').date()]) &
    (
        df_leads_l34['utm_source'].isna() |
        (df_leads_l34['utm_source'].str.lower() == 'tva')
    )
)
df_leads_l34.loc[mask_google, 'utm_source'] = 'google-ads'

In [13]:
df_leads_l34.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28278 entries, 0 to 28277
Data columns (total 20 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   id                         28278 non-null  object
 1   nome                       28278 non-null  object
 2   email                      28278 non-null  object
 3   telefone                   28278 non-null  object
 4   data inscrição lançamento  28278 non-null  object
 5   estado                     22939 non-null  object
 6   idade                      22552 non-null  object
 7   escolaridade               22569 non-null  object
 8   renda                      22732 non-null  object
 9   estado civil               22657 non-null  object
 10  filhos                     22784 non-null  object
 11  escolheu profissão         22844 non-null  object
 12  dificuldade                21428 non-null  object
 13  email captação             22939 non-null  object
 14  telefo

In [14]:
df_leads_l34.isnull().sum()

id                              0
nome                            0
email                           0
telefone                        0
data inscrição lançamento       0
estado                       5339
idade                        5726
escolaridade                 5709
renda                        5546
estado civil                 5621
filhos                       5494
escolheu profissão           5434
dificuldade                  6850
email captação               5339
telefone captação            5341
utm_source                     67
utm_campaign                  551
utm_medium                   1497
utm_content                  2149
utm_term                     4726
dtype: int64

In [15]:
df_leads_l34.drop(columns=['email', 'telefone']).head()

,id,nome,data inscrição lançamento,estado,idade,escolaridade,renda,estado civil,filhos,escolheu profissão,dificuldade,email captação,telefone captação,utm_source,utm_campaign,utm_medium,utm_content,utm_term
0,10360,Ivisson Freitas,2025-05-18,Outro,26 - 35 anos,Ensino Médio Completo,Não estou trabalhando no momento,Casado(a),Não,Estabilidade de emprego,Falta de oportunidades na vida,01ivisson02@gmail.com,71996004228,Facebook-Ads,OPC-L34 | CAPTACAO | CBO | FRIO | BR | 16-05-2025,00 | CONCORRENTES,ads_001_captacao_video_stories_p1_imagem_sobral_1,Instagram_Stories
1,9911,Luciano,2025-05-18,Outro,46 - 55 anos,Outro,"De R$ 1.000,00 a R$ 3.000,00",Divorciado(a) ou Separado(a),Sim,Gosta da profissão,Financeiro / Dinheiro,022998828178lucianodahira@gmail.com,73981270523,Facebook-Ads,OPC-L34 | CAPTACAO | CBO | MORNO/QUENTE | BR |...,00 | PAGINA DE CAPTURA 180D,ads_001_captacao_video_stories_p1_imagem_sobral_1,Instagram_Stories
2,10583,Sandra,2025-05-18,None,None,None,None,None,None,None,None,None,None,Facebook-Ads,OPC-L34 | CAPTACAO | CBO | FRIO | BR | 16-05-2025,00 | LAL 1% LEADS QUALIFICADOS,ads_005_captacao_video_stories_p1_video_vemai_v1,Facebook_Mobile_Reels
3,10349,Victor oliveira,2025-05-18,None,None,None,None,None,None,None,None,None,None,Facebook-Ads,OPC-L34 | CAPTACAO | CBO | MORNO/QUENTE | BR |...,00 | ENVOLVIMENTO E PAGEVIEW 180D,ads_001_captacao_video_stories_p1_imagem_sobral_1,Instagram_Stories
4,10274,Tony,2025-05-18,São Paulo,46 - 55 anos,Ensino Superior Completo,"Acima de R$ 5.000,00",Casado(a),Sim,Estabilidade de emprego,None,40graufitness@gmail.com,BR +5571981662336,Facebook-Ads,OPC-L34 | CAPTACAO | CBO | FRIO | BR | 16-05-2025,00 | LAL 1% LEADS QUALIFICADOS,ads_001_captacao_video_stories_p1_imagem_sobral_1,Instagram_Stories


In [16]:
# === Configurações ===
NOME_PLANILHA = "Leads L34"
ABA_NOME = "Página1"  # Pode alterar conforme necessário
EMAIL_PESSOAL = "camilobf2@gmail.com"  # <<< Substitua pelo seu e-mail pessoal do Google

# === Resolve o caminho do projeto de forma segura ===
try:
    base_dir = Path(__file__).resolve().parent
except NameError:
    base_dir = Path().resolve()

project_root = base_dir if base_dir.name == "escola_policia" else base_dir.parent

# === Carrega variável de ambiente com caminho da credencial ===
cred_env_path = os.getenv("GOOGLE_CREDENTIALS_PATH")
if not cred_env_path:
    raise ValueError("❌ Variável de ambiente GOOGLE_CREDENTIALS_PATH não foi definida.")

CRED_PATH = project_root / cred_env_path

if not CRED_PATH.exists():
    raise FileNotFoundError(f"❌ Credencial não encontrada em: {CRED_PATH}")
else:
    print(f"✅ Credencial localizada: {CRED_PATH}")

# === Autenticação com Google Sheets ===
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
credenciais = ServiceAccountCredentials.from_json_keyfile_name(str(CRED_PATH), scope)
client = gspread.authorize(credenciais)

# === Abertura ou criação da planilha ===
try:
    planilha = client.open(NOME_PLANILHA)
except gspread.SpreadsheetNotFound:
    print("📄 Planilha não encontrada. Criando nova...")
    planilha = client.create(NOME_PLANILHA)
    planilha.share(EMAIL_PESSOAL, perm_type='user', role='writer')

# === Seleção da aba e escrita do DataFrame ===
aba = planilha.sheet1  # ou: planilha.worksheet(ABA_NOME)
aba.clear()
set_with_dataframe(aba, df_leads_l34)

print("✅ Dados enviados com sucesso para o Google Sheets.")

✅ Credencial localizada: C:\Users\Camilo_Bica\data_science\consultoria\escola_policia\secrets\credenciais_gsheets.json
✅ Dados enviados com sucesso para o Google Sheets.


In [17]:
# Define caminho seguro para o arquivo .parquet
try:
    base_dir = Path(__file__).resolve().parent
except NameError:
    base_dir = Path().resolve()

project_root = base_dir if base_dir.name == "escola_policia" else base_dir.parent
saida_parquet = project_root / "dados" / "leads_l34.parquet"

# Cria a pasta se não existir
saida_parquet.parent.mkdir(parents=True, exist_ok=True)

# Salva o arquivo
df_leads_l34.to_parquet(saida_parquet, index=False)
print(f"✅ Arquivo salvo com sucesso em: {saida_parquet}")

✅ Arquivo salvo com sucesso em: C:\Users\Camilo_Bica\data_science\consultoria\escola_policia\dados\leads_l34.parquet
